In [1]:
# Load Dataset
import pandas as pd
import numpy as np

In [2]:
apex_df = pd.read_csv('data/apex.csv')

In [3]:
#decision trees good at overfitting small data points
# ONE SHOT LEARNING- learn a differentiator rather than a predictor.  
# maximize the difference between good score and bad score
# transfer learning
# combination of both
# metrics, true positive, true negative, sensitivy, specificity, recall

### Decision Tree Classifier

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
y = apex_df['good']
X = apex_df.drop(['good'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
score = clf.score(X_test, y_test)
print('Test Accuracy: ', score)

Test Accuracy:  1.0


### Random Forest Classifer

In [5]:
from sklearn.ensemble import RandomForestClassifier
y = apex_df['good']
X = apex_df.drop(['good'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
clf_rf = RandomForestClassifier()
clf_rf = clf_rf.fit(X_train,y_train)
y_pred_rf = clf_rf.predict(X_test)
score_rf = clf_rf.score(X_test, y_test)
print('Test Accuracy: ', score_rf)

Test Accuracy:  0.75


/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


### Logistic Regression

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
y = apex_df['good']
X = apex_df.drop(['good'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
clf_log = LogisticRegression()
clf_log = clf_log.fit(X_train,y_train)
y_pred_log = clf_log.predict(X_test)
score_log = clf_log.score(X_test, y_test)
print('Test Accuracy: ', score_log)

Test Accuracy:  0.75


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### One Shot Learning

In [7]:
#run this in terminal to install pip:  curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
#run this in terminal to install tensorflow: pip install --upgrade tensorflow
#run this in terminal to install keras: pip install keras
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from keras.layers import Dropout
from tensorflow.keras.layers import Dense

Using TensorFlow backend.


In [8]:
x = apex_df.drop(['good'], axis =1)
y = apex_df['good']
#pd.get_dummies(y).values
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=42,shuffle=True)

# Core data structure in Keras is a model
# The model is an object in which we organize layers

model_k = Sequential() # instantiate empty Sequential model


# model contruction (architecture build computational graph)


model_k.add( Dense(units=64, input_dim = 19, activation='relu'))

model_k.add( Dense(units=1, activation='softmax') )

# For a binary classification problem 
def mean_pred(y_true, y_pred):
    return keras.backend.mean(y_pred)

#simple_sgd = optimizers.SGD(lr=0.01)  
    
model_k.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_k.fit(X_train, y_train, epochs =10, batch_size= 32)

# Evaluate the model Accuracy on test set
score_keras = model_k.evaluate(X_test, y_test, verbose = False)[1]

print('Test Accuracy: ', score_keras)

Train on 15 samples
Epoch 1/10
15/15 [==============================] - 1s 40ms/sample - loss: 2.0332 - accuracy: 0.8667
Epoch 2/10
15/15 [==============================] - 0s 287us/sample - loss: 2.0332 - accuracy: 0.8667
Epoch 3/10
15/15 [==============================] - 0s 477us/sample - loss: 2.0332 - accuracy: 0.8667
Epoch 4/10
15/15 [==============================] - 0s 298us/sample - loss: 2.0332 - accuracy: 0.8667
Epoch 5/10
15/15 [==============================] - 0s 596us/sample - loss: 2.0332 - accuracy: 0.8667
Epoch 6/10
15/15 [==============================] - 0s 365us/sample - loss: 2.0332 - accuracy: 0.8667
Epoch 7/10
15/15 [==============================] - 0s 355us/sample - loss: 2.0332 - accuracy: 0.8667
Epoch 8/10
15/15 [==============================] - 0s 317us/sample - loss: 2.0332 - accuracy: 0.8667
Epoch 9/10
15/15 [==============================] - 0s 341us/sample - loss: 2.0332 - accuracy: 0.8667
Epoch 10/10
15/15 [==============================] - 0s 235us/s

In [9]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(15, 19)
(15,)
(11, 19)
